## User parser agent Tests

In [ ]:
from agents.user_parser_agent import invoke_user_parser_agent

print("Running REMAS: user_parser_agent")
user_input = input("Type your message: ")

invoke_user_parser_agent(user_input=user_input)

Running REMAS: run_user_parser_agent
✅ Upserted 1 points into 'user_agent_listings' without resetting the collection.
✅ Uploaded user query with ID: 6e9fd5b8e158c408a80980e25e049dac to 'user_agent_listings'


'6e9fd5b8e158c408a80980e25e049dac'

In [3]:
from utils.qdrant_connection import count_qdrant
print("user Qdrant count:", count_qdrant("user_agent_listings"))

user Qdrant count: 1001


In [1]:
from utils.qdrant_connection import get_point_by_id
point_id = input("Enter the point ID to retrieve: ")
point = get_point_by_id("user_agent_listings", point_id)
if point:
    print(f"\n📌 Point ID: {point.id}")
    print("Payload:", point.payload)
else:
    print(f"No point found with ID: {point_id}")    
    


📌 Point ID: 6e9fd5b8-e158-c408-a809-80e25e049dac
Payload: {'listing_id': '6e9fd5b8e158c408a80980e25e049dac', 'state': ['Brooklyn', 'Queens'], 'picture_url': None, 'price': 2200, 'bedrooms': 2, 'available_from': 'October', 'soft_attributes': 'pet-friendly, close to public transportation, quiet neighborhood', 'source': 'user_query'}


## Owner parser agent Tests

In [1]:
from agents.owner_parser_agent import invoke_owner_parser_agent

print("Running REMAS: owner_parser_agent")
owner_input = input("Type your message: ")

invoke_owner_parser_agent(owner_input=owner_input)


Running REMAS: owner_parser_agent
✅ Upserted 1 points into 'owner_agent_listings' without resetting the collection.
✅ Uploaded owner listing with ID: 140d392e74131263a9ca770865306117 to 'owner_agent_listings'


'140d392e74131263a9ca770865306117'

In [2]:
from utils.qdrant_connection import count_qdrant
print("owner Qdrant count:", count_qdrant("owner_agent_listings"))

owner Qdrant count: 1008


In [4]:
from utils.qdrant_connection import get_point_by_id
point_id = input("Enter the point ID to retrieve: ")
point = get_point_by_id("owner_agent_listings", point_id)
if point:
    print(f"\n📌 Point ID: {point.id}")
    print("Payload:", point.payload)
else:
    print(f"No point found with ID: {point_id}")    


📌 Point ID: 140d392e-7413-1263-a9ca-770865306117
Payload: {'listing_id': '140d392e74131263a9ca770865306117', 'state': ['Astoria', 'Queens'], 'picture_url': 'https://example.com/apt-photos/astoria2.jpg', 'price': 2400, 'bedrooms': 2, 'available_from': 'October', 'soft_attributes': 'spacious, 5 minutes from the subway, balcony with city views, newly renovated kitchen, in-unit washer/dryer'}


In [10]:
# pront count similatity 
from utils.qdrant_connection import count_qdrant
print("similarity_collection Qdrant count:", count_qdrant("similarity_collection"))     

similarity_collection Qdrant count: 30064


In [11]:
from utils.qdrant_connection import scroll_qdrant
print("Scrolling through similarity_collection:")
list = scroll_qdrant("similarity_collection", limit=10)[0]
for point in list:
    print(f"\n📌 Point : {point}")

Scrolling through similarity_collection:

📌 Point : id='00002dfd-2619-8928-90fc-a885f9bf7e7c' payload={'user_id': 'b4c4af9a-95cf-3b26-d0d3-6cb6285752e8', 'owner_id': 'd9e08424-bc77-656e-72f3-44c488406268', 'score': 0.55240315, 'filter_used': {'collection': 'user_agent_listings', 'state': ['New York City'], 'price_owner': None, 'bedrooms_owner': 5, 'available_from': None}, 'timestamp': 1754906840.3216486} vector=None shard_key=None order_value=None

📌 Point : id='00018f3a-6b1b-7701-9322-70b25d96fae7' payload={'user_id': '3e91e308-97c8-5c72-b42a-b61498983051', 'owner_id': '5fe3c73d-69fa-29d4-8b1b-34f6fab0f107', 'score': 0.6045309, 'filter_used': {'collection': 'user_agent_listings', 'state': ['Crown Heights', 'Brooklyn', 'New York City'], 'price_owner': 2000, 'bedrooms_owner': 1, 'available_from': None}, 'timestamp': 1754906437.3464315} vector=None shard_key=None order_value=None

📌 Point : id='00034545-0053-1cfc-2e2e-670af297d33d' payload={'user_id': '07a040bc-f51e-501d-6d89-9ed5dbf4d75

## matching

In [ ]:

from agents.owner_parser_agent import invoke_owner_parser_agent
from agents.user_parser_agent import invoke_user_parser_agent
from agents.matching_agent import match_for_new_user, match_for_new_owner

# --- 1) Inputs (aligned) ---
owner_input = """
Spacious 2-bedroom apartment in san fransisco, just 5 minutes from the subway.
Includes a balcony with city views, newly renovated kitchen, and in-unit washer/dryer.
$2400 per month, available starting October.
Picture: https://example.com/apt-photos/astoria2.jpg
""".strip()

user_input = """
Looking for a 2-bedroom place in san fransisco, ideally close to the subway.
Quiet building, good for working from home; in-unit laundry is a plus.
My budget is up to $2,500 per month and I'd like to move in by October.
""".strip()

# --- 2) Parse + upload (each returns the new point id) ---
print("➕ Uploading owner listing…")
owner_point_id = invoke_owner_parser_agent(owner_input)
print("Owner point id:", owner_point_id)

print("\n➕ Uploading user query…")
user_point_id = invoke_user_parser_agent(user_input)
print("User point id:", user_point_id)

➕ Uploading owner listing…
✅ Upserted 1 points into 'owner_agent_listings' without resetting the collection.
✅ Uploaded owner listing with ID: 1935037721de772c96a4cb773255090f to 'owner_agent_listings'
Owner point id: 1935037721de772c96a4cb773255090f

➕ Uploading user query…
✅ Upserted 1 points into 'user_agent_listings' without resetting the collection.
✅ Uploaded user query with ID: 0ebe37acc5ac1c909601d12f03a526f2 to 'user_agent_listings'
User point id: 0ebe37acc5ac1c909601d12f03a526f2


In [18]:
# --- 3) Match both directions ---
TOP_K = 10

print("\n🔎 Matching for NEW USER → owner listings …")
user_matches = match_for_new_user(user_point_id, top_k=TOP_K)

print("🔎 Matching for NEW OWNER → user queries …")
owner_matches = match_for_new_owner(owner_point_id, top_k=TOP_K)


🔎 Matching for NEW USER → owner listings …
🔎 Matching for NEW OWNER → user queries …


In [ ]:
# --- 4) Helper to find rank of counterpart in results ---
def rank_of(target_id: str, results: list) -> tuple[int | None, float | None]:
    for idx, r in enumerate(results, start=1):
        # results from matching_agent are dicts with 'user_id' or 'owner_id'
        other_id = r.get("owner_id") or r.get("user_id")
        if str(other_id) == str(target_id):
            return idx, r.get("score")
    return None, None

# --- 4) Helper to find rank of counterpart in results ---
def rank_of(target_id: str, results: list) -> tuple[int | None, float | None]:
    for idx, r in enumerate(results, start=1):
        # results from matching_agent are dicts with 'user_id' or 'owner_id'
        other_id = r.get("owner_id") or r.get("user_id")
        if str(other_id) == str(target_id):
            return idx, r.get("score")
    return None, None

# --- 5) Evaluate & print ---
def print_top(results, label, key):
    print(f"\nTop {min(len(results), 5)} {label}:")
    for r in results[:5]:
        print(f"  {key}={r.get(key)}  score={round(r.get('score', 0.0), 4)}")

In [21]:
print_top(user_matches,  "owners for this user", "owner_id")
print_top(owner_matches, "users for this owner", "user_id")


Top 1 owners for this user:
  owner_id=19350377-21de-772c-96a4-cb773255090f  score=0.7138

Top 1 users for this owner:
  user_id=0ebe37ac-c5ac-1c90-9601-d12f03a526f2  score=0.7138


In [22]:
u_rank, u_score = rank_of(owner_point_id, user_matches)
o_rank, o_score = rank_of(user_point_id, owner_matches)


In [24]:

print("\n📊 Verification:")
if u_rank:
    print(f"  Owner appears in USER→OWNERS at rank #{u_rank} (score={round(u_score, 4)})")
else:
    print("  Owner NOT found in USER→OWNERS top list – consider relaxing filters or raising TOP_K.")

if o_rank:
    print(f"  User appears in OWNER→USERS at rank #{o_rank} (score={round(o_score, 4)})")
else:
    print("  User NOT found in OWNER→USERS top list – consider relaxing filters or raising TOP_K.")

print("\n✅ Smoke test completed.")




📊 Verification:
  Owner NOT found in USER→OWNERS top list – consider relaxing filters or raising TOP_K.
  User NOT found in OWNER→USERS top list – consider relaxing filters or raising TOP_K.

✅ Smoke test completed.
